# Evolutionary Prompt Selection
## Hyper-parameter Optimization

### Install Dependencies

In [1]:
!printf 'accelerate\nbitsandbytes\ndatasets\npinecone-client[grpc]\nsentencepiece\nsentence-transformers\ntorch\ntransformers\nwikipedia ' > requirements.txt  
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=85d2fc6bbbff3607078e79cad86c53bb57ff4c0ebed7e66cb1fc4576c7aaaaea
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=aec2ee412c9ece381bfe41ce1fb8b89511242

### Import Statements

In [2]:
import json
import math
import os
import string
import time

import pandas as pd
import pinecone
import torch


from accelerate import Accelerator, notebook_launcher
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import GenerationConfig
from tqdm.auto import tqdm

from utils import LanguageModel, EPS, PWS
from nodes import Extractor

from kaggle_secrets import UserSecretsClient


/opt/conda/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEE

### Define Global Variables

In [3]:
user_secrets = UserSecretsClient()
PINECONE_API_KEY = user_secrets.get_secret('PINECONE_API_KEY')
PINECONE_ENV = user_secrets.get_secret('PINECONE_ENVIRONMENT')
INDEX_NAME = 'plans'

EMBEDDING_MODEL = 'all-MiniLM-L6-v2'

MODEL_PATH = "stabilityai/StableBeluga-13B"
SYSTEM_TAG = "### System:\n"
USER_TAG = "### User:\n"
AI_TAG = "### Assistant:\n"

LOAD_IN_8BIT = True
HF_TOKEN = None

TEMPERATURE = 0.01
TOP_K = 50
TOP_P = 0.9
REPETITION_PENALTY= 1.0
MAX_NEW_TOKENS = 256

DATASET_NAME = "trivia_qa"

NUM_EXAMPLES = 3


### Initialize models and agents

In [4]:
generation_config = GenerationConfig(
    do_sample=True,
    temperature=TEMPERATURE,
    top_k=TOP_K,
    top_p=TOP_P,
    repetition_penalty=REPETITION_PENALTY,
    max_new_tokens=MAX_NEW_TOKENS
)

model = LanguageModel(MODEL_PATH, generation_config=generation_config,
                      load_in_8bit=LOAD_IN_8BIT, access_token=HF_TOKEN, device_map='auto',
                      system_tag=SYSTEM_TAG, user_tag=USER_TAG, ai_tag=AI_TAG)

agent = PWS(model)
extractor = Extractor(model)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Initialize database, prompter and dataset

In [5]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)
index = pinecone.GRPCIndex(INDEX_NAME)
embedding_model = SentenceTransformer(EMBEDDING_MODEL)
prompter = EPS(index, embedding_model)

dataset = load_dataset(DATASET_NAME, 'rc.nocontext')
sanitize = lambda text: text.strip().lower().translate(str.maketrans('', '', string.punctuation))


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/138384 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17944 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17210 [00:00<?, ? examples/s]

Dataset trivia_qa downloaded and prepared to /root/.cache/huggingface/datasets/trivia_qa/rc.nocontext/1.2.0/e73c5e47a8704744fa9ded33504b35a6c098661813d1c2a09892eb9b9e9d59ae. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### 1st Round

In [6]:
hp_opt_size = 30
hp_opt_data = dataset['train'][:hp_opt_size]
temp_values = [0.01, 0.25, 0.5, 0.75, 1.0]
rep_values = [1.0, 1.1, 1.2, 1.3]
results = []
for temp in temp_values:
    for rep in rep_values:
        generation_config = GenerationConfig(
            do_sample=True,
            temperature=temp,
            top_k=TOP_K,
            top_p=TOP_P,
            repetition_penalty=rep,
            max_new_tokens=MAX_NEW_TOKENS
        )
        model.generation_config = generation_config
        em = []
        for question, answer in tqdm(zip(hp_opt_data['question'], hp_opt_data['answer']),
                                     total=hp_opt_size):
            list_of_candidates = [sanitize(alias) for alias in answer["aliases"]]

            selection = prompter.select_examples(question, NUM_EXAMPLES)
            examples = [entry['metadata'] for entry in selection]
            response = agent.run(question, examples)
            answer = sanitize(response['output'])

            if answer not in list_of_candidates:
                extracted = sanitize(extractor(response['output'], question))
                if extracted not in list_of_candidates:
                    em.append(False)
                    continue 
            em.append(True)

        print(f"Temperature: {temp}\nRepetition Penalty: {rep}\nScore: {sum(em)}\n")
        results.append({'temp':temp, 'rep':rep, 'score':sum(em)})
    
with open("results.json", "w") as f:
    json.dump(results, f)
    

  0%|          | 0/30 [00:00<?, ?it/s]

Temperature: 0.01
Repetition Penalty: 1.0
Score: 18



  0%|          | 0/30 [00:00<?, ?it/s]

Temperature: 0.01
Repetition Penalty: 1.1
Score: 18



  0%|          | 0/30 [00:00<?, ?it/s]

...

Temperature: 1.0
Repetition Penalty: 1.1
Score: 14



  0%|          | 0/30 [00:00<?, ?it/s]

Temperature: 1.0
Repetition Penalty: 1.2
Score: 11



  0%|          | 0/30 [00:00<?, ?it/s]

Temperature: 1.0
Repetition Penalty: 1.3
Score: 6



#### Results

In [7]:
with open('results.json', 'r') as f:
    results = json.load(f)

results_df = pd.DataFrame.from_dict(results)
results_df = results_df.rename(columns={'temp': 'Temperature',
                                        'rep': 'Repetition Penalty',
                                        'score': 'EM Score'})
results_df = results_df.pivot(index='Temperature', columns='Repetition Penalty')
results_df


EM Score            
Repetition Penalty      1.0 1.1 1.2 1.3
Temperature                            
0.01                     18  18  12   9
0.25                     17  17  12   6
0.50                     15  16  12  10
0.75                     16  15  12   4
1.00                     18  14  11   6

### 2nd Round

In [8]:
top_3 = sorted(results, key=lambda r: r['score'], reverse=True)[:3]
hp_opt_size = 100
hp_opt_data = dataset['train'][:hp_opt_size]
results_v2 = []
for config in top_3:
    temp = config['temp']
    rep = config['rep']
    generation_config = GenerationConfig(
        do_sample=True,
        temperature=temp,
        top_k=TOP_K,
        top_p=TOP_P,
        repetition_penalty=rep,
        max_new_tokens=MAX_NEW_TOKENS
    )
    model.generation_config = generation_config
    em = []
    for question, answer in tqdm(zip(hp_opt_data['question'], hp_opt_data['answer']),
                                 total=hp_opt_size):
        list_of_candidates = [sanitize(alias) for alias in answer["aliases"]]

        selection = prompter.select_examples(question, NUM_EXAMPLES)
        examples = [entry['metadata'] for entry in selection]
        response = agent.run(question, examples)
        answer = sanitize(response['output'])

        if answer not in list_of_candidates:
            extracted = sanitize(extractor(response['output'], question))
            if extracted not in list_of_candidates:
                em.append(False)
                continue 
        em.append(True)

    print(f"Temperature: {temp}\nRepetition Penalty: {rep}\nScore: {sum(em)}\n")
    results_v2.append({'temp':temp, 'rep':rep, 'score':sum(em)})

with open("results_v2.json", "w") as f:
    json.dump(results_v2, f)


  0%|          | 0/100 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/conda/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Temperature: 0.01
Repetition Penalty: 1.0
Score: 49



  0%|          | 0/100 [00:00<?, ?it/s]

Temperature: 0.01
Repetition Penalty: 1.1
Score: 49



  0%|          | 0/100 [00:00<?, ?it/s]

Temperature: 1.0
Repetition Penalty: 1.0
Score: 47

